In [1]:
# %pip install pyodbc pandas faker

In [ ]:
import pyodbc
from faker import Faker
import random
from datetime import datetime, timedelta
import pandas as pd

fake = Faker(['es_MX'])

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=./testdb.accdb;'
)


In [ ]:
def crear_tablas(conn):
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE Usuarios (
            UsuarioID COUNTER PRIMARY KEY,
            Nombre TEXT(100),
            Correo TEXT(100),
            Telefono TEXT(20),
            Direccion TEXT(200),
            CodigoPostal TEXT(5),
            FechaRegistro DATETIME
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Paseadores (
            PaseadorID COUNTER PRIMARY KEY,
            Nombre TEXT(100),
            Correo TEXT(100),
            Telefono TEXT(20),
            Direccion TEXT(200),
            CodigoPostal TEXT(5),
            EstadoVerif TEXT(20),
            Calificacion DOUBLE,
            FechaIngreso DATETIME
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Perros (
            PerroID COUNTER PRIMARY KEY,
            UsuarioID LONG,
            Nombre TEXT(50),
            Raza TEXT(50),
            Tamanio TEXT(20),
            Edad INTEGER,
            NecesidadesEsp TEXT(255)
        )
    ''')
    
    # Crear tabla Servicios
    cursor.execute('''
        CREATE TABLE Servicios (
            ServicioID COUNTER PRIMARY KEY,
            NombreServ TEXT(50),
            Duracion INTEGER,
            PrecioBase CURRENCY,
            Descripcion TEXT(255)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Paseos (
            PaseoID COUNTER PRIMARY KEY,
            UsuarioID LONG,
            PaseadorID LONG,
            PerroID LONG,
            ServicioID LONG,
            HoraInicio DATETIME,
            Estado TEXT(20),
            DuracionReal INTEGER,
            Precio CURRENCY
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Resenias (
            ReseniaID COUNTER PRIMARY KEY,
            PaseoID LONG,
            Calificacion INTEGER,
            Comentario TEXT(255),
            FechaResenia DATETIME
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Disponibilidad (
            DisponibilidadID COUNTER PRIMARY KEY,
            PaseadorID LONG,
            DiaSemana INTEGER,
            HoraInicio DATETIME,
            HoraFin DATETIME
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE Pagos (
            PagoID COUNTER PRIMARY KEY,
            PaseoID LONG,
            Monto CURRENCY,
            FechaPago DATETIME,
            EstadoPago TEXT(20),
            MetodoPago TEXT(50)
        )
    ''')
    
    conn.commit()


In [4]:
def generar_usuarios(n=50):
    usuarios = []
    for _ in range(n):
        usuarios.append((
            fake.name(),
            fake.email(),
            fake.phone_number(),
            fake.street_address(),
            fake.postcode()[:5],
            fake.date_time_between(start_date='-1y')
        ))
    return usuarios

def generar_paseadores(n=30):
    paseadores = []
    estados_verificacion = ['Verificado', 'Pendiente', 'En Revision']
    for _ in range(n):
        paseadores.append((
            fake.name(),
            fake.email(),
            fake.phone_number(),
            fake.street_address(),
            fake.postcode()[:5],
            random.choice(estados_verificacion),
            round(random.uniform(4.0, 5.0), 1),
            fake.date_time_between(start_date='-2y')
        ))
    return paseadores

def generar_perros(usuarios, n=70):
    razas = ['Labrador', 'Chihuahua', 'Pastor Aleman', 'Golden Retriever', 'Bulldog', 'Poodle', 'Husky', 
             'Schnauzer', 'Xoloitzcuintle', 'French Poodle', 'Pit Bull', 'Doberman']
    tamanios = ['Pequeño', 'Mediano', 'Grande']
    perros = []
    for _ in range(n):
        perros.append((
            random.randint(1, len(usuarios)),  # UsuarioID
            fake.first_name(),  # Nombre del perro
            random.choice(razas),
            random.choice(tamanios),
            random.randint(1, 15),  # Edad
            random.choice(['Ninguna', 'Alergias', 'Medicacion diaria', 'Dieta especial', 
                         'Problemas de cadera', 'Ansiedad'])
        ))
    return perros

def generar_servicios():
    servicios = [
        ('Paseo Individual', 30, 150.00, 'Paseo exclusivo para un perro'),
        ('Paseo Grupal', 45, 120.00, 'Paseo en grupo pequeño de hasta 3 perros'),
        ('Paseo Express', 15, 80.00, 'Paseo corto para necesidades basicas'),
        ('Paseo Premium', 60, 250.00, 'Paseo largo con entrenamiento basico'),
        ('Paseo Nocturno', 30, 180.00, 'Paseo especial en horario nocturno')
    ]
    return servicios

def generar_paseos(n=150):
    paseos = []
    estados = ['Programado', 'En Curso', 'Completado', 'Cancelado']
    for _ in range(n):
        servicio_id = random.randint(1, 5)
        precio_base = [150.00, 120.00, 80.00, 250.00, 180.00][servicio_id-1]
        duracion_base = [30, 45, 15, 60, 30][servicio_id-1]
        duracion_real = duracion_base + random.randint(-5, 15)
        ajuste_precio = duracion_real / duracion_base
        precio_final = round(precio_base * ajuste_precio, 2)
        
        paseos.append((
            random.randint(1, 50),  # UsuarioID
            random.randint(1, 30),  # PaseadorID
            random.randint(1, 70),  # PerroID
            servicio_id,
            fake.date_time_between(start_date='-6m'),
            random.choice(estados),
            duracion_real,
            precio_final
        ))
    return paseos

def generar_resenias(n=100):
    resenias = []
    comentarios = [
        "Excelente servicio, muy puntual",
        "Mi perro regreso muy contento",
        "Buen trato y profesionalismo",
        "Se nota que tiene experiencia con perros",
        "Muy recomendable",
        "Regular, llego un poco tarde",
        "Muy buen servicio, repetire",
        "El paseador fue muy amable",
        "Mi mascota la paso increible",
        "Necesita mejorar la puntualidad"
    ]
    
    for _ in range(n):
        resenias.append((
            random.randint(1, 150),  # PaseoID
            random.randint(1, 5),
            random.choice(comentarios),
            fake.date_time_between(start_date='-6m')
        ))
    return resenias

def generar_disponibilidad(n=200):
    disponibilidad = []
    for _ in range(n):
        hora_inicio = random.randint(6, 12)
        hora_fin = random.randint(hora_inicio + 4, 20)
        disponibilidad.append((
            random.randint(1, 30),  # PaseadorID
            random.randint(0, 6),  # DiaSemana (0=Lunes, 6=Domingo)
            datetime.strptime(f"{hora_inicio}:00", "%H:%M").time(),
            datetime.strptime(f"{hora_fin}:00", "%H:%M").time()
        ))
    return disponibilidad

def generar_pagos(n=150):
    pagos = []
    metodos_pago = ['Tarjeta Credito', 'Tarjeta Debito', 'Efectivo', 'Transferencia', 'CoDi']
    estados_pago = ['Completado', 'Pendiente', 'Fallido']
    
    for paseo_id in range(1, n+1):
        monto = random.uniform(80.00, 300.00)
        pagos.append((
            paseo_id,
            round(monto, 2),
            fake.date_time_between(start_date='-6m'),
            random.choice(estados_pago),
            random.choice(metodos_pago)
        ))
    return pagos

In [ ]:
try:
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa a la base de datos")
    
    crear_tablas(conn)
    print("Tablas creadas exitosamente")
    
    cursor = conn.cursor()
    
    usuarios = generar_usuarios(50)
    cursor.executemany(
        "INSERT INTO Usuarios (Nombre, Correo, Telefono, Direccion, CodigoPostal, FechaRegistro) VALUES (?, ?, ?, ?, ?, ?)",
        usuarios
    )
    print("Usuarios insertados")
    
    paseadores = generar_paseadores(30)
    cursor.executemany(
        "INSERT INTO Paseadores (Nombre, Correo, Telefono, Direccion, CodigoPostal, EstadoVerif, Calificacion, FechaIngreso) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        paseadores
    )
    print("Paseadores insertados")
    
    perros = generar_perros(usuarios, 70)
    cursor.executemany(
        "INSERT INTO Perros (UsuarioID, Nombre, Raza, Tamanio, Edad, NecesidadesEsp) VALUES (?, ?, ?, ?, ?, ?)",
        perros
    )
    print("Perros insertados")
    
    servicios = generar_servicios()
    cursor.executemany(
        "INSERT INTO Servicios (NombreServ, Duracion, PrecioBase, Descripcion) VALUES (?, ?, ?, ?)",
        servicios
    )
    print("Servicios insertados")
    
    paseos = generar_paseos(150)
    cursor.executemany(
        "INSERT INTO Paseos (UsuarioID, PaseadorID, PerroID, ServicioID, HoraInicio, Estado, DuracionReal, Precio) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        paseos
    )
    print("Paseos insertados")
    
    resenias = generar_resenias(100)
    cursor.executemany(
        "INSERT INTO Resenias (PaseoID, Calificacion, Comentario, FechaResenia) VALUES (?, ?, ?, ?)",
        resenias
    )
    print("Reseñas insertadas")
    
    disponibilidad = generar_disponibilidad(200)
    cursor.executemany(
        "INSERT INTO Disponibilidad (PaseadorID, DiaSemana, HoraInicio, HoraFin) VALUES (?, ?, ?, ?)",
        disponibilidad
    )
    print("Disponibilidad insertada")
    
    pagos = generar_pagos(150)
    cursor.executemany(
        "INSERT INTO Pagos (PaseoID, Monto, FechaPago, EstadoPago, MetodoPago) VALUES (?, ?, ?, ?, ?)",
        pagos
    )
    print("Pagos insertados")
    
    conn.commit()
    print("Todos los datos insertados exitosamente")
    
except Exception as e:
    print(f"Ocurrió un error: {str(e)}")
finally:
    conn.close()

Conexión exitosa a la base de datos
Tablas creadas exitosamente
Usuarios insertados
Paseadores insertados
Perros insertados
Servicios insertados
Paseos insertados
Reseñas insertadas
Disponibilidad insertada
Pagos insertados
Todos los datos insertados exitosamente
